In [245]:
module cse

using DataStructures: OrderedDict

immutable Cache
    name_to_symbol::Dict{Symbol, Symbol}
    symbol_to_name::Dict{Symbol, Symbol}
    setup::Vector{Expr}
end

Cache() = Cache(Dict{Symbol,Symbol}(), Dict{Symbol,Symbol}(), Vector{Expr}())

function add_element!(cache::Cache, name)
    cache.name_to_symbol[name] = name
    cache.symbol_to_name[name] = name
    name
end

function add_element!(cache::Cache, name, setup::Expr)
    sym = gensym(name)
    cache.name_to_symbol[name] = sym
    cache.symbol_to_name[sym] = name
    push!(cache.setup, :($sym = $(copy(setup))))
    sym
end

cacheify!(setup, expr) = expr

function cacheify!(cache::Cache, expr::Expr)
    if expr.head == :function
        # We can't continue CSE through a function definition, but we can
        # start over inside the body of the function:
        for i in 2:length(expr.args)
            expr.args[i] = cacheify(expr.args[i])
        end
        return expr
    elseif expr.head == :line
        return expr
    elseif expr.head == :call
        available = all([haskey(cache.symbol_to_name, arg) for arg in expr.args])
        if available
            cached_name = Symbol(expr.args...)
            if !haskey(cache.name_to_symbol, cached_name)
                sym = add_element!(cache, cached_name, expr)
            else
                sym = cache.name_to_symbol[cached_name]
            end
            return sym
        end
    end
    for (i, child) in enumerate(expr.args)
        expr.args[i] = cacheify!(cache, child)
    end
    return expr
end

classify_symbols!(available, disqualified, expr, in_assignment) = nothing
classify_symbols!(available, disqualified, sym::Symbol, ::Val{true}) = push!(disqualified, sym)
classify_symbols!(available, disqualified, sym::Symbol, ::Val{false}) = push!(available, sym)

function classify_symbols!(available, disqualified, expr::Expr, in_assignment::Val)
    if expr.head == :line
        # do nothing
    elseif expr.head == :(=)
        # This is an assignment expression, so anything on the left hand
        # side (that is, expr.args[1]), will be marked as disqualified from caching. 
        classify_symbols!(available, disqualified, expr.args[1], Val{true}())
        
        # The remaining arguments are the right hand side of the assignment,
        # so they can still be cached.
        for arg in expr.args[2:end]
            classify_symbols!(available, disqualified, arg, in_assignment)
        end
    elseif expr.head == :function
        # don't recurse further
    else
        for arg in expr.args
            classify_symbols!(available, disqualified, arg, in_assignment)
        end
    end
end
        
function find_input_symbols(expr)
    available = Set{Symbol}()
    disqualified = Set{Symbol}()
    classify_symbols!(available, disqualified, expr, Val{false}())
    setdiff!(available, disqualified)
end
    
function cacheify(expr::Expr)
    cache = Cache()
    for var in find_input_symbols(expr)
        add_element!(cache, var)
    end
    while true
        num_setup = length(cache.setup)
        expr = copy(expr)
        expr = cacheify!(cache, expr)
        if length(cache.setup) == num_setup
            break
        end
    end
    Expr(:block, cache.setup..., expr)
end

macro cse(expr)
    result = cacheify(expr)
    println(result)
    esc(result)
end
    
end

cse

In [246]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[246], line 19:
    function foo(x) # In[246], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [247]:
cse.cacheify(expr)

quote 
    begin  # In[246], line 19:
        function foo(x)
            ##gx#429 = g(x)
            ##f##gx#429#430 = f(##gx#429)
            ##h##f##gx#429#430##gx#429#431 = h(##f##gx#429#430,##gx#429)
            begin  # In[246], line 20:
                [h(##h##f##gx#429#430##gx#429#431,i) == i for i = 1:5]
            end
        end
    end
end

In [248]:
@cse.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#432 = f(x)
        begin  # In[248], line 2:
            [##fx#432 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [249]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [250]:
@cse.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#433 = g(x)
    ##f##gx#433#434 = f(##gx#433)
    ##h##f##gx#433#434##gx#433#435 = h(##f##gx#433#434,##gx#433)
    begin  # In[250], line 2:
        [h(##h##f##gx#433#434##gx#433#435,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false